# Aceleração em Data Science - Semana 7

_"A arte, como a moralidade, consiste em estabelecer um limite em algum lugar." - G.K. Cherterton_

### Engenharia de Features

#### Dados: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

### Setup

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Criação e leitura do Data Frame

In [2]:
df = pd.read_csv('train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
df.sample(5).T

,787,1452,347,710,997
Id,788,1453,348,711,998
MSSubClass,60,180,20,30,20
MSZoning,RL,RM,RL,RL,RL
LotFrontage,76,35,NaN,56,NaN
LotArea,10142,3675,17600,4130,11717
...,...,...,...,...,...
MoSold,1,5,12,7,2
YrSold,2010,2006,2009,2008,2009
SaleType,WD,WD,WD,WD,WD
SaleCondition,Normal,Normal,Normal,Normal,Normal


In [4]:
X_train = df.drop(columns = 'SalePrice')
y_train = df['SalePrice']

In [5]:
X_test = pd.read_csv('test.csv')
y_test = pd.read_csv('sample_submission.csv')

### Análise

##### Seleção por completude

In [6]:
#Criando um dataframe auxliar para analisar a consistencia das variaveis
cons = pd.DataFrame({'colunas' : df.columns,
                    'tipo': df.dtypes,
                    'missing' : df.isna().sum(),
                    'size' : df.shape[0],
                    'unicos': df.nunique()})
cons['percentual'] = round(cons['missing'] / cons['size'],2)

In [7]:
cons.head()

,colunas,tipo,missing,size,unicos,percentual
Id,Id,int64,0,1460,1460,0.00
MSSubClass,MSSubClass,int64,0,1460,15,0.00
MSZoning,MSZoning,object,0,1460,5,0.00
LotFrontage,LotFrontage,float64,259,1460,110,0.18
LotArea,LotArea,int64,0,1460,1073,0.00


### Removendo colunas com dados missing

In [8]:
print('Contagem de colunas com ATÉ 20% de dados faltantes', cons[cons.percentual < 0.2].shape[0])
print('Contagem de colunas com 0% de dados faltantes',  cons[cons.percentual == 0].shape[0])

Contagem de colunas com ATÉ 20% de dados faltantes 76
Contagem de colunas com 0% de dados faltantes 63


In [9]:
cons[cons.percentual == 0]['tipo'].value_counts()

int64     35
object    28
Name: tipo, dtype: int64

In [10]:
cons['completa'] = ['completa' if x == 0 else 'faltante' for x in cons['percentual']]

In [11]:
mantem = list(cons[cons['completa'] == 'completa']['colunas'])
df = df[mantem]

In [12]:
df.drop

<bound method DataFrame.drop of         Id  MSSubClass MSZoning  LotArea Street LotShape LandContour  \
0        1          60       RL     8450   Pave      Reg         Lvl   
1        2          20       RL     9600   Pave      Reg         Lvl   
2        3          60       RL    11250   Pave      IR1         Lvl   
3        4          70       RL     9550   Pave      IR1         Lvl   
4        5          60       RL    14260   Pave      IR1         Lvl   
...    ...         ...      ...      ...    ...      ...         ...   
1455  1456          60       RL     7917   Pave      Reg         Lvl   
1456  1457          20       RL    13175   Pave      Reg         Lvl   
1457  1458          70       RL     9042   Pave      Reg         Lvl   
1458  1459          20       RL     9717   Pave      Reg         Lvl   
1459  1460          20       RL     9937   Pave      Reg         Lvl   

     Utilities LotConfig LandSlope  ... EnclosedPorch 3SsnPorch ScreenPorch  \
0       AllPub    Inside

In [13]:
colunas_numericas = list(cons[((cons['tipo'] != 'object') &
                              (cons['completa'] == 'completa'))]['colunas'])

colunas_numericas.remove('SalePrice')

In [14]:
colunas_categoricas = list(cons[((cons['tipo'] == 'object') &
                              (cons['completa'] == 'completa'))]['colunas'])

### Variáveis categóricas

##### Label Encoding

In [15]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageCars', 'GarageArea',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition', 'SalePrice'],
      dtype='object')

##### Label encoder

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
encoding = LabelEncoder()

In [18]:
encoding.fit_transform(df['TotRmsAbvGrd'])

array([6, 4, 4, ..., 7, 3, 4])

##### Dummies

In [19]:
df.shape

(1460, 63)

In [20]:
pd.get_dummies(df).shape

(1460, 222)

##### Catboost

In [21]:
import category_encoders as ce

In [22]:
catb = ce.CatBoostEncoder(cols=colunas_categoricas)

In [23]:
catb.fit_transform(X_train[colunas_categoricas], y_train)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Foundation,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,...,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890
1,194710.597945,194710.597945,194710.597945,194710.597945,194710.597945,180921.195890,194710.597945,180921.195890,180921.195890,194710.597945,...,180921.195890,194710.597945,194710.597945,194710.597945,194710.597945,180921.195890,194710.597945,194710.597945,194710.597945,194710.597945
2,190307.065297,190307.065297,180921.195890,190307.065297,190307.065297,194710.597945,190307.065297,194710.597945,194710.597945,190307.065297,...,194710.597945,190307.065297,190307.065297,190307.065297,190307.065297,194710.597945,190307.065297,190307.065297,190307.065297,190307.065297
3,198605.298973,198605.298973,202210.597945,198605.298973,198605.298973,180921.195890,198605.298973,180921.195890,204307.065297,198605.298973,...,180921.195890,198605.298973,180921.195890,198605.298973,198605.298973,204307.065297,198605.298973,198605.298973,198605.298973,180921.195890
4,186884.239178,186884.239178,181473.731963,186884.239178,186884.239178,181210.597945,186884.239178,180921.195890,188230.298973,186884.239178,...,204307.065297,186884.239178,198605.298973,186884.239178,186884.239178,188230.298973,186884.239178,186884.239178,186884.239178,198605.298973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,191008.321880,181105.930480,164644.954610,180153.548734,180925.939654,176887.998279,179927.498691,192929.458176,184477.799519,181144.751697,...,225239.593811,181998.639885,214852.720535,186175.352826,186781.834858,139864.757430,183429.208693,186423.144608,173349.107835,175152.475038
1456,190994.377348,181101.722396,164656.185679,180149.608712,180921.869640,176886.198471,179923.925450,188651.728711,184470.259503,181140.490427,...,149775.505057,181993.728559,142294.935037,186167.147721,186772.989637,139912.625607,145447.457932,186414.600745,173350.413921,175152.347444
1457,191010.918360,181121.624791,164705.312238,180172.412678,180941.827176,176917.735425,179945.719707,208946.709723,184490.553415,181160.490087,...,151696.032648,182013.368300,214798.938186,186184.633306,186790.414250,212089.378113,183423.001617,186432.228099,173379.385926,175181.484278
1458,191076.561040,181180.384856,164815.479649,180238.311600,181000.509051,177002.970691,180008.394783,146019.507537,184555.691974,181219.589471,...,149870.646439,182072.574069,157059.517483,186243.515540,122609.619105,212182.229003,183484.132594,186492.024792,173452.940913,175257.773764


### Variáveis númericas

##### Standard Scaler

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
std = StandardScaler()

In [26]:
std.fit_transform(X_train[colunas_numericas])

array([[-1.73086488,  0.07337496, -0.20714171, ..., -0.08768781,
        -1.5991111 ,  0.13877749],
       [-1.7284922 , -0.87256276, -0.09188637, ..., -0.08768781,
        -0.48911005, -0.61443862],
       [-1.72611953,  0.07337496,  0.07347998, ..., -0.08768781,
         0.99089135,  0.13877749],
       ...,
       [ 1.72611953,  0.30985939, -0.14781027, ...,  4.95311151,
        -0.48911005,  1.64520971],
       [ 1.7284922 , -0.87256276, -0.08016039, ..., -0.08768781,
        -0.8591104 ,  1.64520971],
       [ 1.73086488, -0.87256276, -0.05811155, ..., -0.08768781,
        -0.1191097 ,  0.13877749]])

##### MinMaxScaler

In [27]:
from sklearn.preprocessing import MinMaxScaler

In [28]:
minmax = MinMaxScaler()

In [29]:
minmax.fit_transform(X_train[colunas_numericas])

array([[0.00000000e+00, 2.35294118e-01, 3.34198042e-02, ...,
        0.00000000e+00, 9.09090909e-02, 5.00000000e-01],
       [6.85400960e-04, 0.00000000e+00, 3.87950174e-02, ...,
        0.00000000e+00, 3.63636364e-01, 2.50000000e-01],
       [1.37080192e-03, 2.35294118e-01, 4.65072799e-02, ...,
        0.00000000e+00, 7.27272727e-01, 5.00000000e-01],
       ...,
       [9.98629198e-01, 2.94117647e-01, 3.61868705e-02, ...,
        1.61290323e-01, 3.63636364e-01, 1.00000000e+00],
       [9.99314599e-01, 0.00000000e+00, 3.93418869e-02, ...,
        0.00000000e+00, 2.72727273e-01, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 4.03701886e-02, ...,
        0.00000000e+00, 4.54545455e-01, 5.00000000e-01]])

##### MinAbsScaler

In [30]:
from sklearn.preprocessing import MaxAbsScaler

In [31]:
maxabs = MaxAbsScaler()

In [32]:
maxabs.fit_transform(X_train[colunas_numericas])

array([[6.84931507e-04, 3.15789474e-01, 3.92575902e-02, ...,
        0.00000000e+00, 1.66666667e-01, 9.99004975e-01],
       [1.36986301e-03, 1.05263158e-01, 4.46003391e-02, ...,
        0.00000000e+00, 4.16666667e-01, 9.98507463e-01],
       [2.05479452e-03, 3.15789474e-01, 5.22660224e-02, ...,
        0.00000000e+00, 7.50000000e-01, 9.99004975e-01],
       ...,
       [9.98630137e-01, 3.68421053e-01, 4.20079444e-02, ...,
        1.61290323e-01, 4.16666667e-01, 1.00000000e+00],
       [9.99315068e-01, 1.05263158e-01, 4.51439058e-02, ...,
        0.00000000e+00, 3.33333333e-01, 1.00000000e+00],
       [1.00000000e+00, 1.05263158e-01, 4.61659969e-02, ...,
        0.00000000e+00, 5.00000000e-01, 9.99004975e-01]])